In [1]:
import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')

In [3]:
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [4]:
# g.draw()

In [5]:
bc = nx.betweenness_centrality(g)
cc = nx.closeness_centrality(g)
ec = nx.eigenvector_centrality(g, max_iter=1000)

In [6]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'betweenness': [i for i in bc.values()],
    'closeness': [i for i in cc.values()],
    'eigenvector': [i for i in ec.values()]
})

In [7]:
char_infos = pd.read_csv('dataset/charcters_stats.csv', index_col=0)

In [8]:
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [9]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [11]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')
semi_df

,id,Name,ConnAmnt,betweenness,closeness,eigenvector,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
0,1009220,Captain America,565.0,0.025503,0.787805,0.101860,good,63,19,35,56,46,100,319
1,1010740,Winter Soldier,229.0,0.001859,0.591575,0.058647,good,56,32,35,65,60,84,332
2,1009471,Nick Fury,411.0,0.014581,0.695371,0.087608,good,75,11,23,42,25,100,276
3,1009515,Punisher,276.0,0.003916,0.618182,0.067632,good,50,16,23,28,22,100,239
4,1009535,Red Skull,237.0,0.002087,0.598703,0.062822,bad,75,10,12,14,19,80,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1009322,Goblin Queen,61.0,0.000037,0.507463,0.019152,bad,75,10,23,28,65,56,257
168,1009217,Callisto,84.0,0.000107,0.524777,0.026552,bad,63,53,23,42,45,85,311
169,1009150,Agent Zero,63.0,0.000063,0.519710,0.019570,good,75,28,38,80,72,95,388
170,1009160,Arclight,58.0,0.000031,0.515152,0.018464,bad,38,63,23,42,52,70,288


In [12]:
import statsmodels.api as sm

model = sm.OLS(semi_df['betweenness'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            betweenness   R-squared (uncentered):                   0.306
Model:                            OLS   Adj. R-squared (uncentered):              0.293
Method:                 Least Squares   F-statistic:                              24.68
Date:                Wed, 11 Nov 2020   Prob (F-statistic):                    2.79e-13
Time:                        15:59:38   Log-Likelihood:                          639.42
No. Observations:                 171   AIC:                                     -1273.
Df Residuals:                     168   BIC:                                     -1263.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence  7.331e-05   1.89e-05      3.871      0.000    3.59e-05       0.000
Strength      -4.81e-07   1.56e-05     -0.031      0.975   -3.12e-05    3.02e-05
Power        -1.431e-05   1.96e-05     -0.729      0.467   -5.31e-05    2.44e-05
==============================================================================
Omnibus:                      141.910   Durbin-Watson:                   1.128
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1472.685
Skew:                           3.154   Prob(JB):                         0.00
Kurtosis:                      15.919   Cond. No.                         5.52
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
model = sm.OLS(semi_df['closeness'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              closeness   R-squared (uncentered):                   0.838
Model:                            OLS   Adj. R-squared (uncentered):              0.835
Method:                 Least Squares   F-statistic:                              288.9
Date:                Wed, 11 Nov 2020   Prob (F-statistic):                    4.59e-66
Time:                        15:59:39   Log-Likelihood:                          1.8965
No. Observations:                 171   AIC:                                      2.207
Df Residuals:                     168   BIC:                                      11.63
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0071      0.001      9.016      0.000       0.006       0.009
Strength         0.0006      0.001      0.861      0.391      -0.001       0.002
Power            0.0013      0.001      1.607      0.110      -0.000       0.003
==============================================================================
Omnibus:                        8.142   Durbin-Watson:                   1.438
Prob(Omnibus):                  0.017   Jarque-Bera (JB):                8.341
Skew:                           0.541   Prob(JB):                       0.0154
Kurtosis:                       3.019   Cond. No.                         5.52
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
model = sm.OLS(semi_df['eigenvector'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            eigenvector   R-squared (uncentered):                   0.765
Model:                            OLS   Adj. R-squared (uncentered):              0.761
Method:                 Least Squares   F-statistic:                              182.0
Date:                Wed, 11 Nov 2020   Prob (F-statistic):                    1.49e-52
Time:                        15:59:39   Log-Likelihood:                          368.05
No. Observations:                 171   AIC:                                     -730.1
Df Residuals:                     168   BIC:                                     -720.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0007   9.26e-05      7.830      0.000       0.001       0.001
Strength      5.676e-07   7.61e-05      0.007      0.994      -0.000       0.000
Power         8.866e-05    9.6e-05      0.924      0.357      -0.000       0.000
==============================================================================
Omnibus:                        2.975   Durbin-Watson:                   1.490
Prob(Omnibus):                  0.226   Jarque-Bera (JB):                2.089
Skew:                          -0.070   Prob(JB):                        0.352
Kurtosis:                       2.477   Cond. No.                         5.52
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""